# i. Imports and load the data 

In [33]:
import re
from pathlib import Path
import os

from matplotlib.ticker import FuncFormatter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import PolynomialFeatures
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
import nfl_data_py as nfl

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

n_cores = os.cpu_count()

In [141]:
train_data_path = Path('../data/processed/field_goal_data.parquet')
fg_attempts = (
    pd.read_parquet(train_data_path)
    .assign(
        iced_kicker=lambda x: x['iced_kicker'].astype(int)
    )
    .merge(
        pd.read_csv('../data/processed/stadium_elevations.csv')
            [['stadium_id','elevation_feet']],
        on='stadium_id',
        how='left'
    )
    .drop_duplicates()
)
fg_attempts.head()

,season,week,game_id,play_id,game_date,start_time,time_of_day,qtr,quarter_seconds_remaining,game_seconds_remaining,stadium,stadium_id,is_home,is_indoor,roof,grass_surface,lighting_condition,posteam,defteam,posteam_score,defteam_score,score_differential,kicker_player_name,kicker_player_id,kicker_headshot_url,kicker_height,kicker_weight,kicker_years_exp,kicker_draft_number,kicker_age,kicker_rich_hill,temperature,chance_of_rain,snow_severity,wind_speed,wind_gust,yardline_100,tie_or_take_lead,to_stay_within_one_score,pressure_rating,iced_kicker,field_goal_result,elevation_feet
0,2010,1,2010_01_GB_PHI,660.0,2010-09-12,"9/12/10, 16:15:46",2010-09-12T20:37:49Z,1.0,268.0,2968.0,Lincoln Financial Field,PHI00,1,0,outdoors,1,0,PHI,GB,0.0,0.0,0.0,David Akers,00-0000108,https://static.www.nfl.com/image/private/f_aut...,70.0,200.0,13.0,300.0,35.759,0.5,69.0,0.0,0,4.0,4.0,27.0,1,0,0.0,0,made,33
1,2010,1,2010_01_GB_PHI,3652.0,2010-09-12,"9/12/10, 16:15:46",2010-09-12T23:14:33Z,4.0,346.0,346.0,Lincoln Financial Field,PHI00,1,0,outdoors,1,0,PHI,GB,17.0,27.0,-10.0,David Akers,00-0000108,https://static.www.nfl.com/image/private/f_aut...,70.0,200.0,13.0,300.0,35.759,0.5,69.0,0.0,0,4.0,4.0,5.0,0,1,1.0,0,made,33
2,2010,4,2010_04_WAS_PHI,865.0,2010-10-03,"10/3/10, 16:15:32",2010-10-03T20:54:58Z,2.0,900.0,2700.0,Lincoln Financial Field,PHI00,1,0,outdoors,1,0,PHI,WAS,0.0,14.0,-14.0,David Akers,00-0000108,https://static.www.nfl.com/image/private/f_aut...,70.0,200.0,13.0,300.0,35.817,0.5,63.0,0.0,0,13.0,13.0,31.0,0,0,0.0,0,made,33
3,2010,4,2010_04_WAS_PHI,1680.0,2010-10-03,"10/3/10, 16:15:32",2010-10-03T21:35:05Z,2.0,23.0,1823.0,Lincoln Financial Field,PHI00,1,0,outdoors,1,0,PHI,WAS,3.0,17.0,-14.0,David Akers,00-0000108,https://static.www.nfl.com/image/private/f_aut...,70.0,200.0,13.0,300.0,35.817,0.5,63.0,0.0,0,13.0,13.0,6.0,0,0,0.0,0,made,33
4,2010,5,2010_05_PHI_SF,986.0,2010-10-10,"10/10/10, 20:30:42",2010-10-11T01:09:05Z,2.0,705.0,2505.0,Candlestick Park,SFO00,-1,0,outdoors,1,2,PHI,SF,7.0,7.0,0.0,David Akers,00-0000108,https://static.www.nfl.com/image/private/f_aut...,70.0,200.0,13.0,300.0,35.836,0.5,71.0,0.0,0,10.0,10.0,14.0,1,0,0.0,0,made,35


In [143]:
fg_attempts.field_goal_result.value_counts(normalize=False)

made       13325
missed      2152
blocked      310
Name: field_goal_result, dtype: int64

# ii. Load the trained model

In [144]:
model_path = Path('../models/trace_poly2_v1.nc')
if model_path.exists():
    trace = az.from_netcdf(model_path)
    print("Trace loaded successfully.")
else:
    print(f"Model file {model_path} does not exist. Please check the path or run the model first.")

Trace loaded successfully.


# iii. Define the dataset of "replacement" kickers

In [145]:
YEARS = range(2010, 2025)
data_root = Path('../data/raw/weekly_rosters')
data_root.mkdir(parents=True, exist_ok=True)

def load_or_cache_weekly_rosters(year):
    cache_file = data_root / f"{year}.parquet"
    if cache_file.exists():
        print(f"Loading weekly rosters for {year} from cache...")
        return pd.read_parquet(cache_file)
    else:
        print(f"Downloading weekly rosters for {year} from NFL API...")
        df = nfl.import_weekly_rosters([year])
        df.to_parquet(cache_file, index=False)
        print(f"Cached weekly rosters for {year} to {cache_file}")
        return df
def load_all_weekly_rosters(years):
    dataframes = []
    
    for year in years:
        df = load_or_cache_weekly_rosters(year)
        dataframes.append(df)
    
    print(f"Combining weekly rosters for {len(dataframes)} seasons...")
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(f"Total rows: {len(combined_df):,}")
    
    return combined_df
weekly_rosters = load_all_weekly_rosters(YEARS).query('position=="K"')
weekly_rosters.head()

Loading weekly rosters for 2010 from cache...
Loading weekly rosters for 2011 from cache...
Loading weekly rosters for 2012 from cache...
Loading weekly rosters for 2013 from cache...
Loading weekly rosters for 2014 from cache...
Loading weekly rosters for 2015 from cache...
Loading weekly rosters for 2016 from cache...
Loading weekly rosters for 2017 from cache...
Loading weekly rosters for 2018 from cache...
Loading weekly rosters for 2019 from cache...
Loading weekly rosters for 2020 from cache...
Loading weekly rosters for 2021 from cache...
Loading weekly rosters for 2022 from cache...
Loading weekly rosters for 2023 from cache...
Loading weekly rosters for 2024 from cache...
Combining weekly rosters for 15 seasons...
Total rows: 610,092


,season,team,position,depth_chart_position,jersey_number,status,player_name,first_name,last_name,birth_date,height,weight,college,player_id,espn_id,sportradar_id,yahoo_id,rotowire_id,pff_id,pfr_id,fantasy_data_id,sleeper_id,years_exp,headshot_url,ngs_position,week,game_type,status_description_abbr,football_name,esb_id,gsis_it_id,smart_id,entry_year,rookie_year,draft_club,draft_number,age
21,2010,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,13.0,https://static.www.nfl.com/image/private/f_aut...,None,17,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,36.066
22,2010,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,13.0,https://static.www.nfl.com/image/private/f_aut...,None,9,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,35.912
23,2010,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,13.0,https://static.www.nfl.com/image/private/f_aut...,None,16,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,36.052
24,2010,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,13.0,https://static.www.nfl.com/image/private/f_aut...,None,13,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,35.981
25,2010,PHI,K,None,2,ACT,David Akers,David,Akers,1974-12-09,70.0,200.0,None,00-0000108,None,None,None,None,None,None,None,None,13.0,https://static.www.nfl.com/image/private/f_aut...,None,11,REG,A01,David,AKE551610,None,3200414b-4555-1610-e0e6-a72c82e419e7,1997.0,1997.0,None,None,35.951


In [183]:
active_weekly_kickers.head()

,season,week,team,status,kicker_player_name,kicker_player_id,rookie_year,years_exp,attempts
0,2010,1,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0
1,2010,2,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0
2,2010,3,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0
3,2010,4,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0
4,2010,5,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,3.0


In [184]:
weekly_kick_attempts.head()

,season,week,team,kicker_player_name,kicker_player_id,attempts
0,2010,1,ARI,Jay Feely,00-0019770,1
1,2010,1,ATL,Matt Bryant,00-0020578,4
2,2010,1,BAL,Billy Cundiff,00-0020972,1
3,2010,1,BUF,Rian Lindell,00-0019310,2
4,2010,1,CAR,John Kasay,00-0009028,3


In [187]:
fg_attempts.kicker_years_exp.value_counts()

1.0     1369
2.0     1356
0.0     1303
3.0     1259
6.0     1208
5.0     1164
4.0     1151
8.0     1000
7.0      962
9.0      834
10.0     799
11.0     731
12.0     555
14.0     458
13.0     387
15.0     387
16.0     247
17.0     179
19.0     125
20.0     111
18.0     109
21.0      34
23.0      31
22.0      28
Name: kicker_years_exp, dtype: int64

In [ ]:
team_mapper = {
    'BLT': 'BAL',
    'OAK': 'LV',
    'SD': 'LAC',
    'SL': 'LA',
    'HST': 'HOU',
    'ARZ': 'ARI',
    'CLV': 'CLE',
}
active_weekly_kickers = (
    weekly_rosters
    [['season','week','team','status','player_name','player_id','rookie_year','years_exp']]
    .rename(columns={
        'player_name': 'kicker_player_name',
        'player_id': 'kicker_player_id'
    })
    .sort_values(['season','team','week'])
    # .query('status=="ACT"')
    .assign(
        team=lambda x: x['team'].replace(team_mapper),
    )
)


# Add the number of kick attempts for each kicker in each week
weekly_kick_attempts = (
    fg_attempts
    [['season','week','posteam','kicker_player_name','kicker_years_exp','kicker_player_id']]
    .rename(columns={
        'posteam': 'team',
    })
    .groupby(['season','week','team','kicker_player_name','kicker_years_exp','kicker_player_id'])
    .aggregate({'kicker_player_id': 'count'})
    .rename(columns={
        'kicker_player_id': 'attempts',
    })
    .query('attempts > 0')
    .reset_index()
    .rename(columns={
        'kicker_years_exp': 'years_exp',
    })
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        weekly_kick_attempts.drop(columns=['kicker_player_name', 'years_exp']),
        on=['season','week','team','kicker_player_id'],
        how='left'
    )
    .fillna({'attempts': 0})
    # .query('attempts > 0')
)
active_weekly_kickers = (
    pd.concat([
        active_weekly_kickers
            .assign(
                original_df=True
            ),
        weekly_kick_attempts
            .assign(
                status='ACT',
                rookie_year=lambda x: x['season'] - x['years_exp'],
                original_df=False
            )
            [list(active_weekly_kickers.columns) + ['original_df']]
    ])
    .sort_values(['season','team', 'week', 'kicker_player_id','original_df'], ascending=[True, True, True, True, False])
    .drop_duplicates(['season','team','kicker_player_id','week'])
    .drop(columns=['original_df'])
    .reset_index(drop=True)
    .assign(
        rookie_season=lambda x: x['rookie_year'] == x['season'],
        second_season=lambda x: x['rookie_year'] + 1 == x['season'],
        third_season=lambda x: x['rookie_year'] + 2 == x['season'], 
        fourth_season=lambda x: x['rookie_year'] + 3 == x['season'],
    )
)


# Find the starting kicker for each team in week 1 of each season
week_one_starting_kickers = (
    fg_attempts
    .sort_values(['season','week','game_id','posteam','game_seconds_remaining'], ascending=[True, True, True, True, False])
    .drop_duplicates(['season','posteam'])
    [['season','posteam','kicker_player_id']]
    .rename(columns={
        'posteam': 'team',
        'kicker_player_id': 'season_starting_kicker_player_id'
    })
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        week_one_starting_kickers,
        on=['season','team'],
        how='left'
    )
    .assign(
        season_starting_kicker=lambda x: x['kicker_player_id'] == x['season_starting_kicker_player_id']
    )
    .drop(columns=['season_starting_kicker_player_id'])
)


# Find out if the kicker was on the active roster in week 1 of that season
week_one_kickers =(
    active_weekly_kickers
    .query('week==1')
    [['season','team','kicker_player_id']]
    .rename(columns={
        'kicker_player_id': 'kicker_player_id_week_one'
    })
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        week_one_kickers,
        on=['season','team'],
        how='left'
    )
    .assign(
        on_active_roster_week_one=lambda x: x['kicker_player_id'] == x['kicker_player_id_week_one'],
    )
    .sort_values(['season','team','kicker_player_name','week', 'on_active_roster_week_one'], ascending=[True, True, True, True, False])
    .drop_duplicates(['season','team','kicker_player_id','week'])
    .drop(columns=['kicker_player_id_week_one'])
)


# Add column to indicate if the kicker was the starting kicker in week 1 of the next season
week_one_starting_kickers_next_season = (
    week_one_starting_kickers
    .rename(columns={
        'season_starting_kicker_player_id': 'season_starting_kicker_player_id_next_season'
    })
    .assign(
        season=lambda x: x['season'] - 1.
    )
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        week_one_starting_kickers_next_season,
        on=['season','team'],
        how='left'
    )
    .assign(
        season_starting_kicker_next_season=lambda x: x['kicker_player_id'] == x['season_starting_kicker_player_id_next_season']
    )
    .drop(columns=['season_starting_kicker_player_id_next_season'])
)


# Find out if they kicked for the team they kicked for the team the previous season
active_kickers_previous_season = (
    active_weekly_kickers
    # .query('attempts > 0')
    [['season','team','kicker_player_id']]
    .rename(columns={
        'team': 'team_previous_season'
    })
    .assign(
        season=lambda x: x['season'] + 1
    )
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        active_kickers_previous_season,
        on=['season','kicker_player_id'],
        how='left'
    )
    .assign(
        kicked_for_team_previous_season=lambda x: x['team'] == x['team_previous_season']
    )
    .sort_values(['season','team','kicker_player_name','week','kicked_for_team_previous_season'], ascending=[True, True, True, True, False])
    .drop_duplicates(['season','team','kicker_player_id','week'])
    .drop(columns=['team_previous_season'])
)


# Number of fg attemps by a third and fourth year kicker on the same team
third_season_attempts = (
    active_weekly_kickers
    .query('third_season')
    .groupby(['season','team','kicker_player_id'])
    .aggregate({'attempts': 'sum'})
    .reset_index()
    .rename(columns={
        'attempts': 'attempts_third_season'
    })
)
fourth_season_attempts = (
    active_weekly_kickers
    .query('fourth_season')
    .groupby(['season','team','kicker_player_id'])
    .aggregate({'attempts': 'sum'})
    .reset_index()
    .rename(columns={
        'attempts': 'attempts_fourth_season'
    })
)
active_weekly_kickers = (
    active_weekly_kickers
    .merge(
        third_season_attempts,
        on=['season','team','kicker_player_id'],
        how='left'
    )
    .merge(
        fourth_season_attempts,
        on=['season','team','kicker_player_id'],
        how='left'
    )
    .fillna({'attempts_third_season': 0, 'attempts_fourth_season': 0})
)

active_weekly_kickers.head()

,season,week,team,status,kicker_player_name,kicker_player_id,rookie_year,years_exp,attempts,rookie_season,second_season,third_season,fourth_season,season_starting_kicker,on_active_roster_week_one,season_starting_kicker_next_season,kicked_for_team_previous_season,attempts_third_season,attempts_fourth_season
0,2010,1,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0,False,False,False,False,True,True,True,False,0.0,0.0
1,2010,2,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0,False,False,False,False,True,True,True,False,0.0,0.0
2,2010,3,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0,False,False,False,False,True,True,True,False,0.0,0.0
3,2010,4,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,1.0,False,False,False,False,True,True,True,False,0.0,0.0
4,2010,5,ARI,ACT,Jay Feely,00-0019770,2001.0,9.0,3.0,False,False,False,False,True,True,True,False,0.0,0.0


In [211]:
replacement_kicker_seasons = (
    pd.concat([
        # CASE 1: player picked up mid-season and not retained as the starting kicker the next season
        active_weekly_kickers.query(
            '~season_starting_kicker and ' # Not the starting kicker for the team that season
            '~on_active_roster_week_one and '  # Not on active roster in week 1
            '~season_starting_kicker_next_season and '  # Not the same teams starting kicker the next season
            '~kicked_for_team_previous_season'  # Did not kick for the same team the previous season
        ),
        # CASE 2: rookie kicker who is no longer an active team starter after 2 seasons
        active_weekly_kickers.query(
            '(rookie_season or second_season) and '  # Rookie season or second season
            ' attempts_third_season < 10 and '  # Did not register at least 10 kicks with the team in their 3rd season
            ' attempts_fourth_season < 10'  # Did not register at least 10 kicks with the team in their 4th season
        )
    ])
)[['season','team','kicker_player_id','kicker_player_name']].drop_duplicates().reset_index(drop=True)

replacement_kicker_seasons.shape

(365, 4)

In [213]:
replacement_kicker_seasons.head()

,season,team,kicker_player_id,kicker_player_name
0,2010,CAR,00-0023363,Rhys Lloyd
1,2010,CIN,00-0027405,Aaron Pettrey
2,2010,CIN,00-0027191,Clint Stitser
3,2010,DAL,00-0001980,Kris Brown
4,2010,DEN,00-0025944,Steven Hauschka


In [218]:
(
    fg_attempts.merge(
        replacement_kicker_seasons.rename(columns={'team':'posteam'}),
        on=['season','posteam','kicker_player_id'],
        how='left',
        suffixes=('', '_replacement')
    )
    .assign(
        replacement_kicker=lambda x: x['kicker_player_name_replacement'].notna()
    )
    .drop(columns=['kicker_player_name_replacement'])
).replacement_kicker.value_counts(normalize=True)

False    0.778489
True     0.221511
Name: replacement_kicker, dtype: float64

In [202]:
active_weekly_kickers.groupby(['season','team']).aggregate({'attempts': 'sum'}).reset_index().attempts.describe()

count    480.000000
mean      32.889583
std        6.557462
min       16.000000
25%       28.000000
50%       32.000000
75%       38.000000
max       56.000000
Name: attempts, dtype: float64

In [203]:
active_weekly_kickers.groupby(['season','team']).aggregate({'attempts': 'sum'}).reset_index().sort_values('attempts', ascending=True).head()

,season,team,attempts
133,2014,CHI,16.0
179,2015,MIA,16.0
190,2015,TEN,16.0
256,2018,ARI,17.0
311,2019,NYG,17.0


In [204]:
fg_attempts[['season','posteam','kicker_player_id','kicker_player_name']].drop_duplicates().shape

(645, 4)

In [212]:
365 / 645

0.5658914728682171

In [ ]:
# Replacement if:
    # CASE 1: player picked up mid-season and not retained as the starting kicker the next season
        # First player to attempt a kick for that team that season
        # Joins roster after week 1 (not on active roster in week 1)
        # Not the same teams starting kicker the next season
        # Did not kick for the same team the previous season
    # CASE 2: rookie kicker who is no longer a team starter after 2 season
        # did not register at least 10 kicks with the team in their 3rd or 4th seasons

In [7]:
import pandas as pd

# Filter the dataset to exclude rookies in 2023+
filtered_fg = (
    fg_attempts
    [['kicker_player_name','kicker_player_id', 'posteam', 'season', 'kicker_years_exp']]
    .query('~(season >= 2023 and kicker_years_exp <= 2)')
    .drop_duplicates(subset=['kicker_player_id', 'posteam', 'season'])
)

# Sort and group
filtered_fg = filtered_fg.sort_values(['kicker_player_id', 'posteam', 'season'])

# Longest streak of consecutive seasons per kicker/team
def longest_consecutive_streak(seasons):
    return (
        seasons
        .sort_values()
        .diff()
        .eq(1)
        .cumsum()
        .max()
    ) + 1

# Compute max successive seasons for each kicker/team
successive_seasons = (
    filtered_fg
    .groupby(['kicker_player_name', 'kicker_player_id', 'posteam'])['season']
    .apply(longest_consecutive_streak)
    .reset_index(name='max_successive_seasons')
)

# Get the max streak per kicker (across teams)
kicker_max_streak = (
    successive_seasons
    .sort_values('max_successive_seasons', ascending=False)
    .drop_duplicates(subset=['kicker_player_id'])
)

# =====================
# Add total seasons played and FG attempt counts
# =====================

# Count number of seasons with a FG attempt
n_seasons_played = (
    fg_attempts
    .groupby(['kicker_player_name', 'kicker_player_id'])['season']
    .nunique()
    .reset_index(name='n_seasons_played')
)

# Count total FG attempts
total_fg_attempts = (
    fg_attempts
    .groupby(['kicker_player_name', 'kicker_player_id'])
    .size()
    .reset_index(name='total_fg_attempts')
)

# Merge all together
kicker_summary = (
    kicker_max_streak
    .merge(n_seasons_played, on=['kicker_player_name', 'kicker_player_id'], how='left')
    .merge(total_fg_attempts, on=['kicker_player_name', 'kicker_player_id'], how='left')
)

# Calculate FG attempts per season
kicker_summary['fg_attempts_per_season'] = (
    kicker_summary['total_fg_attempts'] / kicker_summary['n_seasons_played']
)

# Sort for review
kicker_summary = kicker_summary.sort_values('total_fg_attempts', ascending=False)

kicker_summary.head()

,kicker_player_name,kicker_player_id,posteam,max_successive_seasons,n_seasons_played,total_fg_attempts,fg_attempts_per_season
0,Justin Tucker,00-0029597,BAL,13,13,518,39.846154
12,Matt Prater,00-0023853,DET,7,15,461,30.733333
9,Greg Zuerlein,00-0029621,LA,8,13,429,33.000000
1,Mason Crosby,00-0025580,GB,13,14,414,29.571429
11,Nick Folk,00-0025565,NYJ,7,14,412,29.428571


In [12]:
# kicker_intercepts = fg_attempts.query('2015 <= season')[['kicker_player_name','kicker_player_id', 'season']].drop_duplicates()
# kicker_intercepts['kicker_season_idx'] = kicker_intercepts['kicker_player_id'].astype(str) + "_" + kicker_intercepts['season'].astype(str)

# kicker_intercepts = kicker_intercepts.merge(
#     pd.DataFrame({
#         'kicker_season_idx': kicker_intercepts['kicker_season_idx'],
#         'kicker_intercept': trace.posterior['kicker_season_intercept'].mean(dim='chain').mean(axis=0).values
#     }), 
#     on='kicker_season_idx', 
#     how='left'
# )
# kicker_intercepts.query('season==2024').sort_values('kicker_intercept', ascending=False, ignore_index=True)

In [13]:
# kicker_season_ytg_slopes = fg_attempts.query('2015 <= season')[['kicker_player_name','kicker_player_id', 'season']].drop_duplicates()
# kicker_season_ytg_slopes['kicker_season_idx'] = kicker_season_ytg_slopes['kicker_player_id'].astype(str) + "_" + kicker_season_ytg_slopes['season'].astype(str)

# kicker_season_ytg_slopes = kicker_season_ytg_slopes.merge(
#     pd.DataFrame({
#         'kicker_season_idx': kicker_season_ytg_slopes['kicker_season_idx'],
#         'kicker_season_ytg_slope': trace.posterior['kicker_ytg_slope'].mean(dim='chain').mean(axis=0).values
#     }), 
#     on='kicker_season_idx', 
#     how='left'
# )
# kicker_season_ytg_slopes.query('season==2024').sort_values('kicker_season_ytg_slope', ascending=False, ignore_index=True)